# Climate Suitability

Climate Data Download

We will be using MACAV2 Data which is a combination of historical climate data combined at a high resolution and projected climate data gather from global climate models (this is at a much lower resolution)

There are downscaled datasets available for the continental US. Data was drawn from the Northwest Knowldege Network data sets, which can be found here

Selections were based upon RCP 'Representative Concentration Pathway. I select an RCP of 8.5, "8.5" signifies a high emissions scenario, meaning it projects a future with continued high levels of greenhouse gas emissions, leading to significant global warming by the end of the century, representing one of the most extreme potential climate change outcomes; the higher the RCP number, the more severe the projected climate change (Australian Government Department for the Environment and Energy, n.d.). RCP8.5 emissions showing a continued high level of greenhouse gas emissions. This is the most severe projection.

Two scenarios were selected for this project:

    - Precipitation - precipitation was selected as Kentucky Coffeetrees struggle to thrive in standing water or soil that is poorly drained combined with high precipitation
    - Kentucky Coffeetrees thrives best in warm temperatures between the 80s to 90s F.

Dataset2 Name for precip variable 1: http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2021_2025_CONUS_monthly.nc.html

Dataset1 Name for Precip variable 2: REACCH MACAV2-METDATA BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2076_2080_CONUS_monthly.nc
References:

    Australian Government Department for the Environment and Energy(n.d.), What are the RPCs?, Accessed on 15 Dec 2024, URL: https://coastadapt.com.au/sites/default/files/infographics/15-117-NCCARFINFOGRAPHICS-01-UPLOADED-WEB%2827Feb%29.pdf
    MACAV2 Data Set Creator, Dr. John Abatzoglou, University of Idaho, reacch@uidaho.edu, http://www.reacchpna.org/



In [4]:
# Add necessary packages
import pathlib
import numpy
import os
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import xarray as xr #because it is a heirarchial data format (.nc)
import matplotlib.pyplot as plt

In [2]:
%store -r crowley_gdf louisa_gdf data_dir srtm_da 

In [5]:
# always build up your paths using os and pathlib so it is reproducable

### Site Directory
site_dir = os.path.join(data_dir, 'crowley_climate')
os.makedirs(site_dir, exist_ok = True)

# Step 4: Confirm creation
print(f"Data directory created at: {site_dir}")



Data directory created at: /Users/erinzimmerman/earth-analytics/data2025/habitat_suitability/crowley_climate


In [ ]:
# Define a function to convert longitude from 0-360 to -180 to 180
def convert_longitude(longitude):
    """Convert longitude range from 0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_tasmin_BNU-ESM_r1i1p1_rcp85_2076_2080_CONUS_monthly.nc

In [6]:
maca_da_list = []
for site_name, site_gdf in {
    'Crowley Ridge': crowley_gdf, 
    'Port Louisa National Wildlife Refuge': louisa_gdf}.items():
        for variable in ['pr']:
            for start_year in [2010, 2070]:
                end_year = start_year + 4

                # Define the URL (not relevant to this specific question, but for completeness)
                maca_url = (
                    'http://thredds.northwestknowledge.net:8080/'
                    'thredds/dodsC/MACAV2/BNU-ESM/'
                    f'macav2metdata_{variable}_BNU-ESM_r1i1p1_rcp85_'
                    f'{start_year}_{end_year}_CONUS_monthly.nc')

                # Open the dataset
                maca_da = xr.open_dataset(maca_url).squeeze().precipitation

                # Set the boundary
                bounds_maca = site_gdf.to_crs(maca_da.rio.crs).total_bounds

                # Correct longitude to be from -180 to 180
                maca_da = maca_da.assign_coords(
                    lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
                )

                # Set spatial dimensions of maca_da
                maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

                # Crop Maca_da
                maca_da = maca_da.rio.clip_box(*bounds_maca)
                maca_da_list.append(dict(
                     site_names=site_name,
                     variable=variable,
                     start_year=start_year,
                     da=maca_da
                ))
# Display the updated dataset
maca_df = pd.DataFrame(maca_da_list)



OSError: [Errno -90] NetCDF: file not found: 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2010_2014_CONUS_monthly.nc'

In [ ]:
maca_df.drop('da', axis='columns')